# KubeVirt - Extract Data from virt-launcher pods
When running a virtualmachine in Kubernetes (and OpenShift) the libvirtd and qemu process run in a container.  
This notebook extracts the logging data out of the pod.

In [ ]:
import yaml
import ipywidgets as widgets
import time
import kubernetes
from kubernetes.client.rest import ApiException
from kubernetes.stream import stream
import openshift
import urllib3
import IPython
from IPython.core.display import display, HTML
from pygments import highlight
from pygments.lexers import XmlLexer
from pygments.formatters import HtmlFormatter
# Disable the annoying ssl warnings
urllib3.disable_warnings()

### Authentication
The `kubevirt-privileged` token can be used to run this notebook:
```
oc serviceaccounts get-token kubevirt-privileged -n kube-system
```

In [ ]:
auth_form_item_layout = widgets.Layout(display='flex',flex_flow='row',justify_content='space-between')

token = widgets.Textarea(value='')
namespace = widgets.Text(value='virtualmachine')
kube = widgets.Text(value='https://kubernetes.default.svc')
auth_button = widgets.Button(description="Connect")

form_items= [
    widgets.Box([widgets.Label(value='Token'),
                token],
                layout=auth_form_item_layout),
    widgets.Box([widgets.Label(value='Kubernetes URL'), 
                 kube],
                layout=auth_form_item_layout),
    widgets.Box([widgets.Label(value='Namespace'), 
                 namespace],
                layout=auth_form_item_layout),
    widgets.Box([widgets.Label(value=''),
                 auth_button],
                layout=auth_form_item_layout),
    
]
auth_form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='80%'
))


auth_form

In [ ]:
def auth_on_button_clicked(b):
    kubernetes_set_configuration()
    kubernetes_connect_to_api()
    display(create_extract_form())

auth_button.on_click(auth_on_button_clicked)

In [ ]:
def kubernetes_set_configuration():
    configuration = kubernetes.client.Configuration()
    configuration.api_key_prefix['authorization'] = 'Bearer'
    configuration.api_key['authorization'] = token.value
    configuration.assert_hostname = False
    setattr(configuration, 'verify_ssl', False)
    setattr(configuration, 'host', kube.value)
    kubernetes.client.Configuration.set_default(configuration)
    print("Kubernetes API configured.")

In [ ]:
def kubernetes_connect_to_api():
    global v1
    global custom
    try:
        v1 = kubernetes.client.CoreV1Api()
        custom = kubernetes.client.CustomObjectsApi()
        print("Kubernetes API objects available.")
    except Exception as e:
        print("Unknown Error: {0}".format(e))

In [ ]:
def display_xml_nice(xml):
    try:
        formatter = HtmlFormatter()
        IPython.display.display(HTML('<style type="text/css">{}</style>    {}'.format(formatter.get_style_defs('.highlight'), 
                                                                  highlight(xml, XmlLexer(), formatter))))
    except Exception as e:
        print("XML: {0}".format(e))

In [ ]:
def kubernetes_pod_exec(name, namespace, container, command):
    stdout = ''
    stderr = ''
    
    try:
        exec_command = ['/bin/sh', '-c', command]
        resp = stream(v1.connect_get_namespaced_pod_exec, 
                      name, 
                      namespace,
                      command=exec_command,
                      stderr=True, stdin=True,
                      stdout=True, tty=False,
                      _preload_content=False)
        
        while resp.is_open():
            resp.update(timeout=5)
            if resp.peek_stdout():
                stdout = resp.read_stdout()
                break
            if resp.peek_stderr():
                stderr = resp.read_stderr()
                print("Command Error {0}".format(stderr))
                break
            else:
                time.sleep(5)
        
        resp.close()
        return stdout
    except ApiException as e:
        print("Kubernetes Connection API Error: {0}".format(e))
    except Exception as e:
        print("Unknown Error: {0}".format(e))

In [ ]:
def kubevirt_virtualmachine_instances():
    try:
        group = "kubevirt.io"
        version = "v1alpha2"
        virtualmachinenames = []
        virtualmachineinstances = custom.list_cluster_custom_object(group, 
                                                                version, 
                                                                'virtualmachineinstances', 
                                                                async_req=False)
        return virtualmachineinstances['items']
    except Exception as e:
        print("Unknown Error: {0}".format(e))
    

In [ ]:
def kubevirt_virtualmachines():
    try:
        group = "kubevirt.io"
        version = "v1alpha2"
        virtualmachines = custom.list_cluster_custom_object(group, 
                                                        version, 
                                                        'virtualmachines', 
                                                        async_req=False)
        return virtualmachines['items']
    except Exception as e:
        print("Unknown Error: {0}".format(e))
    

In [ ]:
def kubevirt_virtualmachine_cluster_ip(virtualmachine_name, namespace):
    try:
        pods = v1.list_namespaced_pod(namespace=namespace, 
                                      label_selector="kubevirt.io/domain={0}".format(virtualmachine_name))
        return pods.items[0].status.pod_ip

    except Exception as e:
        print("Unknown Error: {0}".format(e))
    

In [ ]:
def kubevirt_extract_data_from_pod(virtualmachine_name, namespace):
    commands = []
    try:
        pods = v1.list_namespaced_pod(namespace=namespace, 
                                      label_selector="kubevirt.io/domain={0}".format(virtualmachine_name))
        
        commands.append("virsh dumpxml {0}_{1}".format(namespace, virtualmachine_name) )
        commands.append("cat /var/log/libvirt/qemu/{0}_{1}.log".format(namespace, virtualmachine_name))
        

        for c in commands:
            stdout = kubernetes_pod_exec(pods.items[0].metadata.name, namespace, "compute", c)
            
            if c.find("xml"):
                display_xml_nice(stdout)
                
    except Exception as e:
        print("Unknown Error: {0}".format(e))

In [ ]:
def kubevirt_create_ansible_inventory(ansible_user, ansible_ssh_pass):
    global inventory
    inventory = "[kubevirt]\n"
    try:
        virtualmachine_instances = kubevirt_virtualmachine_instances()
    
    
        for v in virtualmachine_instances:
            ip = kubevirt_virtualmachine_cluster_ip(v['metadata']['name'], namespace.value)
            inventory += "{0} ansible_host={1} ansible_user={2} ansible_ssh_pass={3}\n".format(
                v['metadata']['name'], 
                ip,
                ansible_user,
                ansible_ssh_pass
            )
    except Exception as e:
        print("Unknown Error: {0}".format(e))
    
    %store inventory > hosts.ini

In [ ]:
def extract_on_button_clicked(b):
    
    kubevirt_create_ansible_inventory("cloud-user", "r3dh@t2018")
    
    try:
        for v in extract_select.value:
            kubevirt_extract_data_from_pod(v, namespace.value)
    except Exception as e:
        print("Unknown Error: {0}".format(e))
    
def create_extract_form():
    global extract_select
    instances = []
    extract_form_item_layout = widgets.Layout(display='flex',flex_flow='row',justify_content='space-between')
    extract_button = widgets.Button(description="Execute")
    extract_button.on_click(extract_on_button_clicked)
    virtualmachine_instances = kubevirt_virtualmachine_instances()
    
    for v in virtualmachine_instances:
        instances.append(v['metadata']['name'])
    
    extract_select = widgets.SelectMultiple(options=instances)


    form_items= [
        widgets.Box([widgets.Label(value='Virtual Machines'),
                extract_select],
                layout=extract_form_item_layout),
        widgets.Box([widgets.Label(value=''),
                     extract_button],
                    layout=extract_form_item_layout),
    ]
    extract_form = widgets.Box(form_items, layout=widgets.Layout(
        display='flex',
        flex_flow='column',
        border='solid 2px',
        align_items='stretch',
        width='80%'))
    
    return extract_form

## Sources
- https://github.com/kubernetes-client/python/blob/master/examples/exec.py
- https://stackoverflow.com/questions/40085818/jupyter-notebook-output-cell-syntax-highlighting